In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import shutil

##Data Loading, folders creation and image moving

In [ ]:
#download images and annotations
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2022-04-27 11:45:54--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  29.4MB/s    in 26s     

2022-04-27 11:46:21 (28.6 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]



In [ ]:
#unzip images and annotations
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

In [ ]:
IMG_DIR = "/content/images"
#Creating folders 
IMG_DIR_DOGS = "dogs"
IMG_DIR_CATS = "cats"
new_folder_dogs = os.path.join(IMG_DIR, IMG_DIR_DOGS)
new_folder_cats = os.path.join(IMG_DIR, IMG_DIR_CATS)

#creating a folder for dog class
if not os.path.exists(new_folder_dogs):
  os.makedirs(new_folder_dogs)

#creating a folder for cat class
if not os.path.exists(new_folder_cats):
  os.makedirs(new_folder_cats)

In [ ]:

#moving images to dog or cat folder depeding on the first char
for img_dir in os.listdir(IMG_DIR):
  if img_dir != IMG_DIR_DOGS and img_dir != IMG_DIR_CATS: #excluding folders
    is_uppercase = img_dir[0].isupper()
    if not is_uppercase: #dog image
      old_img_dir = os.path.join(IMG_DIR, img_dir)
      new_img_dir = os.path.join(new_folder_dogs, img_dir)
      shutil.move(old_img_dir, new_img_dir)
    elif is_uppercase: #cat image
      old_img_dir = os.path.join(IMG_DIR, img_dir)
      new_img_dir = os.path.join(new_folder_cats, img_dir)
      shutil.move(old_img_dir, new_img_dir)


In [ ]:
def create_multiclass_imagefolder(path_to_dataset, mkdirs=False):
  ann_dir = os.path.join(path_to_dataset, "annotations")
  full_path = os.path.join(path_to_dataset, "img_dir")
  # Create class-folders

  with open(ann_dir + "/list.txt") as f:
      lines = f.readlines()
      classes = []
      for l in lines:
          full_name = l.split()[0]
          class_name = '_'.join(full_name.split('_')[:-1])
          if class_name not in classes:
              classes.append(class_name)
      # First time only
      if mkdirs:
          for class_name in classes:
            os.mkdir(os.path.join(full_path, class_name))
  # move files
  for root, dirs, files in os.walk(full_path):
      if root == full_path:
          for file in files:
              class_name = '_'.join(file.split('_')[:-1])
              shutil.move(os.path.join(full_path,file),os.path.join(full_path,class_name,file))

In [ ]:
image_datasets = datasets.ImageFolder(IMG_DIR)

print(image_datasets)

Dataset ImageFolder
    Number of datapoints: 7390
    Root location: /content/images


In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=image_datasets, batch_size=8, shuffle=True, num_workers=4)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for i in range(10):
        batch_x, batch_y = next(iter(data_loader))
        print(np.shape(batch_x), batch_y)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored